In [4]:
from sklearn.neighbors import KDTree
import numpy as np
import random
import dask.array as da
import napari
import pandas as pd

%gui qt

coord_path = 'C:/Users/cameron.trader/OneDrive - Allen Institute/Documents/Unet/0539050069-0035_r.csv'

coord = np.loadtxt(coord_path, skiprows=1, delimiter=',')

In [5]:
zarr_img_path = 'C:/Users/cameron.trader/OneDrive - Allen Institute/Documents/Unet/0539050069-0035.zarr'
image = da.from_zarr(zarr_img_path)

In [23]:
coord_df = pd.DataFrame(coord)
coord_df.columns = ['x', 'y', 'size']
size_over_50 = coord_df['size'] >= 50
coord_df = coord_df[size_over_50]
coord_s50 = coord_df.to_numpy()
type(coord_s50)

numpy.ndarray

In [30]:
kdt = KDTree(coord_s50[:,:2], leaf_size=30, metric='euclidean')

close_points = []
close_set =[]
indices_to_drop = []

close = kdt.query_radius(coord_s50[:,:2], r=6) 
for i in close:
    if i.size > 1:
        close_points.append(list(i))
for i in close_points:
    if not i in close_set:
        close_set.append(i)
print("Close points are: ", str(close_set))
for i in close_set:
    while len(i)>1:
        index = random.randrange(len(i))
        indices_to_drop.append(i[index])
        del i[index]

print(str(len(indices_to_drop)),"points to be removed:", str(indices_to_drop))
# print("These points are located at:")
# np.set_printoptions(suppress=True)
# for i in indices_to_drop:
#     print(coord[indices_to_drop])
# np.set_printoptions(suppress=False)
coord_s50_clean = np.array([i for j, i in enumerate(coord_s50) if j not in indices_to_drop])
coord_s50_removed = np.array([i for j, i in enumerate(coord_s50) if j in indices_to_drop])

print(str(coord_s50.shape[0]-coord_s50_clean.shape[0]), "points removed")

Close points are:  [[9348, 7202], [7288, 7504], [9639, 9908], [13061, 11672], [14919, 14433]]
5 points to be removed: [7202, 7288, 9908, 11672, 14919]
5 points removed


In [33]:
viewer = napari.view_image(image[:, :, 0], name='slice')
viewer.add_points(coord_s50_clean[:, :2], size=5, face_color='red')
viewer.add_points(coord_s50_removed[:, :2], size=5, face_color='yellow')

C:\Users\cameron.trader\Anaconda3\envs\napari\lib\site-packages\napari\_vispy\vispy_image_layer.py:198: UserWarning: data shape (30000, 40000) exceeds GL_MAX_TEXTURE_SIZE 32768 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


<Points layer 'Points [1]' at 0x1e3cdb0dbe0>

In [276]:
np.savetxt('/Users/elyse.morin/OneDrive - Allen Institute/Transgenic_Technology_Elyse/Projects/Unet_test_output/547656/csv/0539050069-0035_r_clean.csv', coord_clean, delimiter=",")

np.savetxt('/Users/elyse.morin/OneDrive - Allen Institute/Transgenic_Technology_Elyse/Projects/Unet_test_output/547656/csv/0539050069-0035_r_removed.csv', coord_removed, delimiter=",")